# Liver Model Construction: Notebook

In [1]:
import os
import re
import warnings
from cobra.io.json import load_json_model as load_json_cobra_model
import escher
import mass
import numpy as np
import pandas as pd
import sympy as sym
from cobra import Model, Reaction, Metabolite
import cobra.test
from os.path import join
from mass.util import qcqa
from mass.util import qcqa_model
from cobra import DictList
from mass import (
    MassConfiguration, MassMetabolite, MassModel,
    MassReaction, Simulation, UnitDefinition)
from mass.io.json import save_json_model as save_json_mass_model
from mass.visualization import plot_comparison, plot_time_profile
from mass.visualization import (
    plot_ensemble_phase_portrait, plot_ensemble_time_profile)
mass_config = MassConfiguration()
mass_config.irreversible_Keq = float("inf")
print("MASSpy version: {0}".format(mass.__version__))
from six import iteritems
import matplotlib.pyplot as plt
from mass.thermo import (
    ConcSolver, sample_concentrations,
    update_model_with_concentration_solution)
from cobra.io.json import *
import cobra_dict as c_d
import csv


MASSpy version: 0.1.1


In [2]:
maps_dir = os.path.abspath("../massters_model/maps")
data_dir = os.path.abspath("../massters_model/data")
extra_dir=os.path.abspath("../massters_model")

## Add the pathway metabolites and their information to a new COBRA model

In [3]:
# Initiate empty model
trial= cobra.Model()


In [4]:
#Function to load data from the excel sheet
def load_data(filepath, sheet_name):
    """Load Liver data from an excel sheet"""
    df = pd.read_excel(engine='openpyxl',
        io=filepath,
        sheet_name=sheet_name,
        index_col=0)
    return df

In [5]:
# Loading reaction and metabolite data from Excel file, converting them to dictionaries
# data_sheet_met=os.path.join(data_dir,"metabolite_info.xlsx")

# met_info = load_data(
#     filepath=data_sheet_met,
#     sheet_name="Sheet1")
# # .drop("notes", axis=1).drop("annotation", axis=1)
# metabolite_info=met_info.to_dict(orient='index')



# data_sheet_rxn=os.path.join(data_dir,"reactions_info.xlsx")
# rxn_info = load_data(
#     filepath=data_sheet_rxn,
#     sheet_name="Sheet1")
# reactions_info=rxn_info.to_dict(orient='index')

In [6]:
# making dataframe 
csv_rxn = os.path.join(data_dir,"rxn_df")
csv_met = os.path.join(data_dir,"met_df")
rxn_csv_df = pd.read_csv(csv_rxn) 
met_csv_df = pd.read_csv(csv_met,index_col="id")
met_csv_df=met_csv_df.drop("Unnamed: 0",axis=1)
metabolite_info=met_csv_df.to_dict(orient='index')
met_csv_df

,name,charge,formula,compartment
id,,,,
adp_c,ADP C10H12N5O10P2,-3,C10H12N5O10P2,c
atp_c,ATP C10H12N5O13P3,-4,C10H12N5O13P3,c
g6p_c,D-Glucose 6-phosphate,-2,C6H11O9P,c
glc__D_c,D-Glucose,0,C6H12O6,c
h_c,H+,0,H,c
f6p_c,D-Fructose 6-phosphate,-2,C6H11O9P,c
fdp_c,"D-Fructose 1,6-bisphosphate",-4,C6H10O12P2,c
h2o_c,H2O H2O,0,H2O,c
pi_c,Phosphate,0,HO4P,c


In [7]:
# Loading reaction data as JSON file to maintain data types as dictionaries 
with open(r'/home/mass_user/massters_model/data/rxn') as json_file:
    rxn_data = json.load(json_file)

In [8]:
for key, value in metabolite_info.items():
    print(key, value)

adp_c {'name': 'ADP C10H12N5O10P2', 'charge': -3, 'formula': 'C10H12N5O10P2', 'compartment': 'c'}
atp_c {'name': 'ATP C10H12N5O13P3', 'charge': -4, 'formula': 'C10H12N5O13P3', 'compartment': 'c'}
g6p_c {'name': 'D-Glucose 6-phosphate', 'charge': -2, 'formula': 'C6H11O9P', 'compartment': 'c'}
glc__D_c {'name': 'D-Glucose', 'charge': 0, 'formula': 'C6H12O6', 'compartment': 'c'}
h_c {'name': 'H+', 'charge': 0, 'formula': 'H', 'compartment': 'c'}
f6p_c {'name': 'D-Fructose 6-phosphate', 'charge': -2, 'formula': 'C6H11O9P', 'compartment': 'c'}
fdp_c {'name': 'D-Fructose 1,6-bisphosphate', 'charge': -4, 'formula': 'C6H10O12P2', 'compartment': 'c'}
h2o_c {'name': 'H2O H2O', 'charge': 0, 'formula': 'H2O', 'compartment': 'c'}
pi_c {'name': 'Phosphate', 'charge': 0, 'formula': 'HO4P', 'compartment': 'c'}
dhap_c {'name': 'Dihydroxyacetone phosphate', 'charge': -2, 'formula': 'C3H5O6P', 'compartment': 'c'}
g3p_c {'name': 'Glyceraldehyde 3-phosphate', 'charge': -2, 'formula': 'C3H5O6P', 'compartmen

In [9]:
for key, value in rxn_data.items():
    print(key, value)

HEX1: atp_c + glc__D_c --> adp_c + g6p_c + h_c {'id': 'HEX1', 'name': 'Hexokinase (D-glucose:ATP)', 'metabolites': {'adp_c': 1.0, 'atp_c': -1.0, 'g6p_c': 1.0, 'glc__D_c': -1.0, 'h_c': 1.0}, 'lower_bound': 0.0, 'upper_bound': 1000.0, 'gene_reaction_rule': '3098_AT3 or 80201_AT1 or 2645_AT3 or 3098_AT4 or 3099_AT1 or 2645_AT2 or 3098_AT5 or 2645_AT1 or 3098_AT1 or 3101_AT1 or 3098_AT2'}
PGI: g6p_c <=> f6p_c {'id': 'PGI', 'name': 'Glucose-6-phosphate isomerase', 'metabolites': {'f6p_c': 1.0, 'g6p_c': -1.0}, 'lower_bound': -1000.0, 'upper_bound': 1000.0, 'gene_reaction_rule': '2821_AT1'}
PFK: atp_c + f6p_c --> adp_c + fdp_c + h_c {'id': 'PFK', 'name': 'Phosphofructokinase', 'metabolites': {'fdp_c': 1.0, 'adp_c': 1.0, 'atp_c': -1.0, 'f6p_c': -1.0, 'h_c': 1.0}, 'lower_bound': 0.0, 'upper_bound': 1000.0, 'gene_reaction_rule': '5214_AT1 or 5211_AT2 or (5214_AT1 and 5211_AT1) or (5214_AT1 and 5213_AT1) or 5213_AT1 or (5214_AT1 and 5211_AT2) or 5211_AT1 or (5211_AT1 and 5213_AT1) or (5213_AT1 an

### making cobra model

In [10]:
#  Add all the remaining metabolites involved in the pathway
for met_id, info in metabolite_info.items():
    met = cobra.Metabolite(met_id, name=info['name'], formula=info['formula'], 
                           charge=info['charge'], compartment=info['compartment'])
    trial.add_metabolites(met)

In [11]:
for reaction, info in rxn_data.items():
    print(reaction)
    
    reaction_obj = cobra.Reaction(reaction)
    reaction_obj.id=info['id']
    reaction= reaction_obj.id
    reaction_obj.lower_bound = info['lower_bound']
    reaction_obj.upper_bound = info['upper_bound']
    reaction_obj.name = info['name']
    trial.add_reaction(reaction_obj)
    temp=info['metabolites']
    reaction_obj.add_metabolites(temp)


HEX1: atp_c + glc__D_c --> adp_c + g6p_c + h_c
PGI: g6p_c <=> f6p_c
PFK: atp_c + f6p_c --> adp_c + fdp_c + h_c
FBP: fdp_c + h2o_c --> f6p_c + pi_c
FBA: fdp_c <=> dhap_c + g3p_c
TPI: dhap_c <=> g3p_c
GAPD: g3p_c + nad_c + pi_c <=> 13dpg_c + h_c + nadh_c
PGK: 3pg_c + atp_c <=> 13dpg_c + adp_c
PGM: 2pg_c <=> 3pg_c
ENO: 2pg_c <=> h2o_c + pep_c
PYK: adp_c + h_c + pep_c --> atp_c + pyr_c
ATPM: atp_c + h2o_c --> adp_c + h_c + pi_c
DM_nadh: nadh_c --> h_c + nad_c


In [12]:
#View all the reactions that we need build
escher_builder = escher.Builder(
    model=trial,
    map_json=os.path.join(
        maps_dir, ".".join((
            "Glycolysis", "json"))
    ),
    highlight_missing=True)

escher_builder


Builder(highlight_missing=True)

In [13]:
#Mass balance check
for r in trial.reactions:
    print(r.id,  r.check_mass_balance())

HEX1 {'charge': -1.0}
PGI {}
PFK {'charge': -1.0}
FBP {'charge': 2.0}
FBA {}
TPI {}
GAPD {'charge': -3.0}
PGK {}
PGM {}
ENO {}
PYK {'charge': 1.0}
ATPM {'charge': 1}
DM_nadh {'charge': 1}


In [14]:
trial.metabolites.h_c.charge= 1
# trial.metabolites.h_e.charge= 1
trial.metabolites.pi_c.charge= -2
# trial.metabolites.pi_e.charge= -2
# trial.metabolites.pi_m.charge= -2
# trial.metabolites.h_m.charge= 1
# trial.metabolites.pi_r.charge= -2

In [15]:
#Mass balance check
for r in trial.reactions:
    print(r.id,  r.check_mass_balance())

HEX1 {}
PGI {}
PFK {}
FBP {}
FBA {}
TPI {}
GAPD {}
PGK {}
PGM {}
ENO {}
PYK {}
ATPM {}
DM_nadh {'charge': 2}


### Add boundary reactions to act as source and sink for glucose and pyruvate

In [16]:
# for met in ['pyr_e', 'h2o_e', 'h_e', 'pi_e','lac__L_e']:
#     rxn = cobra.Reaction('EX_%s' % met)
#     trial.add_reaction(rxn)
#     rxn.add_metabolites({met: -1})
#     if met == 'glc__D_c':
#         rxn.lower_bound = -1 # by convention negative exchange flux = uptake
#     elif met == 'pyr_c':
#         rxn.lower_bound = 0
#     else:
#         rxn.lower_bound = -1000

## Making MASS model of liver

In [17]:
liver= MassModel("Core Model", array_type='DataFrame', dtype=np.int64)
    

In [18]:
#  Add all the remaining metabolites involved in the pathway
for met_id, info in metabolite_info.items():
    met = MassMetabolite(met_id, name=info['name'], formula=info['formula'], 
                           charge=info['charge'], compartment=info['compartment'])
    liver.add_metabolites(met)

In [19]:
for reaction, info in rxn_data.items():
    print(reaction)
    reaction_obj = MassReaction(reaction)
    reaction_obj.id=info['id']
    reaction= reaction_obj.id
    reaction_obj.lower_bound = info['lower_bound']
    reaction_obj.upper_bound = info['upper_bound']
    reaction_obj.name = info['name']
    liver.add_reactions(reaction_obj)
    temp=info['metabolites']
    reaction_obj.add_metabolites(temp)

HEX1: atp_c + glc__D_c --> adp_c + g6p_c + h_c
PGI: g6p_c <=> f6p_c
PFK: atp_c + f6p_c --> adp_c + fdp_c + h_c
FBP: fdp_c + h2o_c --> f6p_c + pi_c
FBA: fdp_c <=> dhap_c + g3p_c
TPI: dhap_c <=> g3p_c
GAPD: g3p_c + nad_c + pi_c <=> 13dpg_c + h_c + nadh_c
PGK: 3pg_c + atp_c <=> 13dpg_c + adp_c
PGM: 2pg_c <=> 3pg_c
ENO: 2pg_c <=> h2o_c + pep_c
PYK: adp_c + h_c + pep_c --> atp_c + pyr_c
ATPM: atp_c + h2o_c --> adp_c + h_c + pi_c
DM_nadh: nadh_c --> h_c + nad_c


In [20]:
## Creating reactions for gluconeogenesis

G6P = MassReaction(
    "G6P",
    name="glucose 6-phosphatase",
    reversible=False)
G6P.add_metabolites({
    
    liver.metabolites.get_by_id('g6p_c'): -1,
    liver.metabolites.get_by_id('h2o_c'): -1,
    liver.metabolites.get_by_id('pi_c') : 1,
    liver.metabolites.get_by_id('glc__D_c'): 1})



liver.add_reactions([
   G6P ])


## Lehningers Table 14-3 
# the reaction is to be counted twice, 
# because two three-carbon precursors are required to make a molecule of glucose
# liver.reactions.PCm + liver.reactions.PCm 

# liver.reactions.PEPCKm + liver.reactions.PEPCKm
liver.reactions.PYK.reversible=False
for reaction in liver.reactions:
    print(reaction)

HEX1: atp_c + glc__D_c <=> adp_c + g6p_c + h_c
PGI: g6p_c <=> f6p_c
PFK: atp_c + f6p_c <=> adp_c + fdp_c + h_c
FBP: fdp_c + h2o_c <=> f6p_c + pi_c
FBA: fdp_c <=> dhap_c + g3p_c
TPI: dhap_c <=> g3p_c
GAPD: g3p_c + nad_c + pi_c <=> 13dpg_c + h_c + nadh_c
PGK: 3pg_c + atp_c <=> 13dpg_c + adp_c
PGM: 2pg_c <=> 3pg_c
ENO: 2pg_c <=> h2o_c + pep_c
PYK: adp_c + h_c + pep_c --> atp_c + pyr_c
ATPM: atp_c + h2o_c <=> adp_c + h_c + pi_c
DM_nadh: nadh_c <=> h_c + nad_c
G6P: g6p_c + h2o_c --> glc__D_c + pi_c


### Adding exchange reactions

In [22]:
for met in [
#     'co2_m', 'gdp_m', 'gtp_m', 'pi_m', 'hco3_m','h_m','atp_c','atp_m'
    'glc__D_c','h_c','h2o_c','pi_c','pyr_c'
           ]:
    rxn = MassReaction('EX_%s' % met)
    liver.add_reaction(rxn)
    rxn.add_metabolites({met: -1})
    if met == 'glc__D_c':
        rxn.lower_bound = -1 # by convention negative exchange flux = uptake
    elif met == 'pyr_c':
        rxn.lower_bound = 0
    else:
        rxn.lower_bound = -1000
        
# liver.reactions.L_LACt2r.lower_bound=-1
# liver.reactions.PYRt2.lower_bound=-1
# # liver.reactions.GLCt1.lower_bound=-1
# liver.reactions.Ht.lower_bound=-1000
# liver.reactions.H2Ot.lower_bound=-1000
# liver.reactions.PIt.lower_bound=-1000



In [23]:
liver.metabolites.h_c.charge= 1
# liver.metabolites.h_e.charge= 1
liver.metabolites.pi_c.charge= -2
# liver.metabolites.pi_e.charge= -2
# trial.metabolites.pi_m.charge= -2
# trial.metabolites.h_m.charge= 1

In [24]:
# liver.reactions.GLCt1

In [25]:
for r in liver.reactions:
    print(r.id,  r.check_mass_balance())
print(liver.reactions.FBA.check_mass_balance())

HEX1 {}
PGI {}
PFK {}
FBP {}
FBA {}
TPI {}
GAPD {}
PGK {}
PGM {}
ENO {}
PYK {}
ATPM {}
DM_nadh {'charge': 2}
G6P {}
EX_glc__D_c {'C': -6, 'H': -12, 'O': -6}
EX_h_c {'charge': -1, 'H': -1}
EX_h2o_c {'H': -2, 'O': -1}
EX_pi_c {'charge': 2, 'H': -1, 'O': -4, 'P': -1}
EX_pyr_c {'charge': 1, 'C': -3, 'H': -3, 'O': -3}
{}


In [26]:
liver.stoichiometric_matrix

,HEX1,PGI,PFK,FBP,FBA,TPI,GAPD,PGK,PGM,ENO,PYK,ATPM,DM_nadh,G6P,EX_glc__D_c,EX_h_c,EX_h2o_c,EX_pi_c,EX_pyr_c
adp_c,1,0,1,0,0,0,0,1,0,0,-1,1,0,0,0,0,0,0,0
atp_c,-1,0,-1,0,0,0,0,-1,0,0,1,-1,0,0,0,0,0,0,0
g6p_c,1,-1,0,0,0,0,0,0,0,0,0,0,0,-1,0,0,0,0,0
glc__D_c,-1,0,0,0,0,0,0,0,0,0,0,0,0,1,-1,0,0,0,0
h_c,1,0,1,0,0,0,1,0,0,0,-1,1,1,0,0,-1,0,0,0
f6p_c,0,1,-1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
fdp_c,0,0,1,-1,-1,0,0,0,0,0,0,0,0,0,0,0,0,0,0
h2o_c,0,0,0,-1,0,0,0,0,0,1,0,-1,0,-1,0,0,-1,0,0
pi_c,0,0,0,1,0,0,-1,0,0,0,0,1,0,1,0,0,0,-1,0
dhap_c,0,0,0,0,1,-1,0,0,0,0,0,0,0,0,0,0,0,0,0


In [27]:
liver.objective = 'ATPM'
liver.optimize()

,fluxes,reduced_costs
HEX1,0.0,-2.0
PGI,1.0,0.0
PFK,1.0,0.0
FBP,0.0,-2.0
FBA,1.0,0.0
...,...,...
EX_glc__D_c,-1.0,-6.0
EX_h_c,6.0,0.0
EX_h2o_c,0.0,0.0
EX_pi_c,0.0,0.0


In [28]:
liver.metabolites.atp_c.summary()

Percent,Flux,Reaction,Definition
50.00%,2,PGK,3pg_c + atp_c <=> 13dpg_c + adp_c
50.00%,2,PYK,adp_c + h_c + pep_c --> atp_c + pyr_c
Percent,Flux,Reaction,Definition
75.00%,-3,ATPM,atp_c + h2o_c <=> adp_c + h_c + pi_c
25.00%,-1,PFK,atp_c + f6p_c <=> adp_c + fdp_c + h_c


In [29]:
from cobra.flux_analysis import flux_variability_analysis

In [30]:
flux_variability_analysis(liver, liver.reactions[:])

,minimum,maximum
HEX1,0.0,0.0
PGI,1.0,1.0
PFK,1.0,1.0
FBP,0.0,0.0
FBA,1.0,1.0
TPI,1.0,1.0
GAPD,2.0,2.0
PGK,-2.0,-2.0
PGM,-2.0,-2.0
ENO,2.0,2.0


In [31]:
liver.optimize()
liver.summary(fva=0.95)

Metabolite,Reaction,Flux,Range,C-Number,C-Flux
glc__D_c,EX_glc__D_c,1,[0.95; 1],6,100.00%
Metabolite,Reaction,Flux,Range,C-Number,C-Flux
h_c,EX_h_c,-6,[-6; -5.7],0,0.00%
pyr_c,EX_pyr_c,-2,[-2; -1.9],3,100.00%


In [32]:
cobra.flux_analysis.flux_variability_analysis(
    liver, liver.reactions[:], fraction_of_optimum=0.9)

,minimum,maximum
HEX1,0.000000e+00,0.3
PGI,9.000000e-01,1.0
PFK,9.000000e-01,1.3
FBP,0.000000e+00,0.3
FBA,9.000000e-01,1.0
TPI,9.000000e-01,1.0
GAPD,1.800000e+00,2.0
PGK,-2.000000e+00,-1.8
PGM,-2.000000e+00,-1.8
ENO,1.800000e+00,2.0
